## Label Drift

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import LabelDrift

deepchecks - WARNING - You are using deepchecks version 0.17.5, however a newer version is available. Deepchecks is frequently updated with major improvements. You should consider upgrading via the "python -m pip install --upgrade deepchecks" command.


In [2]:
#cat_features_in_natura = ["Curso","UF","UF_Ensino_Medio","Regiao","Cat_Administrativa","Cat_Org_Academica", "Tipo_Escola_EM",
#                          "Modalidade_EM", "Sexo", "Estado_Civil","Nl_Pai","Nl_Mae", "Moradia","Qtde_Pessoas", "Politica_de_Ingresso",
#                          "Situacao_Trabalho", "Qtde_Livros","Horas_de_Estudo"]
#cat_features_modificadas = ["Turno","Cor_Raca","Renda", "Situacao_Financeira", "Bolsa_de_Estudos","Disponibilidade_Professores", "Dominio_Professores",
 #                            "Condicao_Salas","Condicao_Praticas","Plano_de_Ensino","Formacao"]
cat_features_modificadas = ["Cor_Raca", "Renda", "Politica_de_Ingresso"]

In [64]:
ano_alvo = 2017
#enade_treino = pd.read_csv(f"../pre-process_{ano_alvo}_tres_anos.csv")
enade_treino = pd.read_csv(f"../pre-process_{ano_alvo}.csv")
enade_treino = enade_treino[["Cor_Raca"]]
labels_treino = pd.DataFrame(0, index=np.arange(enade_treino.shape[0]), columns=["dummy"]) 
enade_treino = pd.concat([enade_treino, labels_treino], axis=1)
#print(enade_treino.head(5))
#print(enade_treino.tail(5))

In [65]:
enade_treino

,Cor_Raca,dummy
0,Parda,0
1,Parda,0
2,Parda,0
3,Branca,0
4,Parda,0
...,...,...
51004,Branca,0
51005,Branca,0
51006,Branca,0
51007,Branca,0


In [66]:
#result_comparacao = pd.DataFrame()

#handle_latex_form = [
#    feature_name.replace("_", "\_")
#    for feature_name
#    in cat_features_modificadas
#]

#result_comparacao["Caracteristicas"] = handle_latex_form

#for ano in [2014]:
ano_teste = 2021
enade_teste = pd.read_csv(f"../pre-process_{ano_teste}.csv")
enade_teste = enade_teste[["Cor_Raca"]]
labels_teste = pd.DataFrame(1, index=np.arange(enade_teste.shape[0]), columns=["dummy"]) 
enade_teste = pd.concat([enade_teste, labels_teste], axis=1)

In [67]:
enade_teste

,Cor_Raca,dummy
0,Não declarado,1
1,Branca,1
2,Branca,1
3,Amarela,1
4,Branca,1
...,...,...
62026,Parda,1
62027,Branca,1
62028,Preta,1
62029,Parda,1


In [68]:
#    list_drift_score_ano_comp = []

#    for feature in cat_features_modificadas:
train_dataset = Dataset(enade_treino, label="dummy", cat_features=["Cor_Raca"])
test_dataset = Dataset(enade_teste, label="dummy", cat_features=["Cor_Raca"])

check = LabelDrift(ignore_na=True, balance_classes=True)
result = check.run(train_dataset=train_dataset, test_dataset=test_dataset)
result
#        break
    
#        list_drift_score_ano_comp.append(round(result.value["Drift score"], 5))
        
#    result_comparacao[str(ano)] = list_drift_score_ano_comp

In [17]:
result_comparacao

,Caracteristicas,2014
0,Cor\_Raca,0.99998
1,Renda,0.99998
2,Politica\_de\_Ingresso,0.99998


In [135]:
result_comparacao.to_csv(f'./relatorio_drift/label_drift_{ano_alvo}_in_natura.csv',index=False)

In [137]:
result_comparacao.to_csv(f'./relatorio_drift/label_drift_{ano_alvo}_modificado.csv',index=False)

## Feature Drift

In [1]:
from deepchecks.tabular.checks import FeatureDrift
from sklearn.svm import SVC
from scipy import stats
import scipy

deepchecks - WARNING - You are using deepchecks version 0.17.5, however a newer version is available. Deepchecks is frequently updated with major improvements. You should consider upgrading via the "python -m pip install --upgrade deepchecks" command.


### Carregamento dos dados de 2017 e 2021

In [37]:
#enade_treino = pd.read_csv("../concept_drift/tabela_final_2011_tres_anos_treinamento.csv")
#enade_treino = pd.read_csv("../pre-process_2017.csv")
enade_treino = pd.read_csv("../concept_drift/tabela_final_2017_treinamento.csv")

numero_caracteristicas = enade_treino.shape[1] - 1
X_enade_treino = enade_treino.iloc[:, 0:numero_caracteristicas]
y_enade_treino = enade_treino.iloc[:, -1]

#enade_teste = pd.read_csv("../pre-process_2021.csv")
#enade_teste = pd.read_csv("../concept_drift/tabela_final_2021_tres_anos_treinamento.csv")
enade_teste = pd.read_csv("../concept_drift/tabela_final_2021_treinamento.csv")

In [38]:
train_dataset = Dataset(enade_treino, label='Nota_Conceito_Faixa', cat_features=[])
test_dataset = Dataset(enade_teste, label='Nota_Conceito_Faixa', cat_features=[])

model = SVC(probability=True)
model.fit(X_enade_treino, y_enade_treino)

check = FeatureDrift()
result = check.run(train_dataset=train_dataset, test_dataset=test_dataset, model=model, feature_importance_force_permutation=True)

NameError: name 'SVC' is not defined

In [5]:
result

In [72]:
dict_feature_scores = result.value
list_rank_feature_scores = sorted(dict_feature_scores.items(), key = lambda entry: entry[1]["Drift score"])

In [8]:
labels = []
scores = []

#for feature in list_rank_feature_scores[-1000:]:
for feature in list_rank_feature_scores:
    labels.append(feature[0])
    scores.append(round(feature[1]["Drift score"], 3))

#print(labels)
#print(scores)

In [ ]:
scores

In [12]:
result_comparacao = pd.DataFrame()

handle_latex_form = [
    feature_name.replace("_", "\_")
    for feature_name
    in labels
]

result_comparacao["Valores"] = handle_latex_form
result_comparacao["Drift score"] = scores
result_comparacao

,Valores,Drift score
0,Não sei responder.33,0.190
1,Tem renda/Sustenta,0.193
2,Não sei responder.25,0.199
3,Não sei responder.32,0.206
4,Não sei responder.41,0.209
5,Numero\_Faltantes,0.215
6,Numero\_Participantes,0.216
7,Não sei responder.37,0.233
8,Nenhum.2,0.250
9,Não sei responder.26,0.290


In [185]:
result_comparacao.to_csv(f'./relatorio_drift/feature_drift_11_21.csv',index=False)

In [73]:
list(dict_feature_scores.items())[102]

('nulos_Politica_de_Ingresso',
 {'Drift score': 0.06870462076925937,
  'Method': 'Kolmogorov-Smirnov',
  'Importance': None})

In [74]:
# Forma Ingresso
for i in range(102,110):
    print(list(dict_feature_scores.items())[i])

('nulos_Politica_de_Ingresso', {'Drift score': 0.06870462076925937, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Não', {'Drift score': 0.096405654640007, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Critério étnico-racial', {'Drift score': 0.08199724466619507, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Critério de renda', {'Drift score': 0.02729145722406323, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Bolsa de estudos', {'Drift score': 0.028450683248315123, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Combinação de critérios', {'Drift score': 0.08201834406920605, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Critérios diferentes', {'Drift score': 0.045803080512839656, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('nulos_Situacao_Trabalho', {'Drift score': 0.06870462076925937, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})


In [54]:
list(dict_feature_scores.items())[73]

('Até 1,5 SM',
 {'Drift score': 0.059373720072979086,
  'Method': 'Kolmogorov-Smirnov',
  'Importance': None})

In [55]:
# Renda
for i in range(73,81):
    print(list(dict_feature_scores.items())[i])

('Até 1,5 SM', {'Drift score': 0.059373720072979086, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('De 1,5 a 3 SM', {'Drift score': 0.03716101326780108, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('De 3 a 4,5 SM', {'Drift score': 0.15518983257003138, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('De 4,5 a 6 SM', {'Drift score': 0.07208548981643514, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('De 6 a 10 SM', {'Drift score': 0.06009606434076775, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('De 10 a 30 SM', {'Drift score': 0.03184396370902687, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Acima de 30 SM', {'Drift score': 0.01882439090988075, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('nulos_Situacao_Financeira', {'Drift score': 0.06870462076925937, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})


In [38]:
list(dict_feature_scores.items())[45]

('nulos_Cor_Raca',
 {'Drift score': 0.06870462076925937,
  'Method': 'Kolmogorov-Smirnov',
  'Importance': None})

In [39]:
# Raça
for i in range(46,52):
    print(list(dict_feature_scores.items())[i])

('Branca', {'Drift score': 0.041259262017351195, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Preta', {'Drift score': 0.04047610182323669, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Amarela', {'Drift score': 0.08332898933859578, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Parda', {'Drift score': 0.04589368383165157, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Indígena', {'Drift score': 0.03704062255650298, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})
('Não declarado', {'Drift score': 0.06958707227166094, 'Method': 'Kolmogorov-Smirnov', 'Importance': None})


## Multivariate Drift

In [5]:
from deepchecks.tabular.checks import MultivariateDrift

In [33]:
enade_treino = pd.read_csv("../concept_drift/tabela_final_2011_treinamento.csv")
#enade_treino = pd.read_csv("../pre-process_2011.csv")
#enade_treino = pd.read_csv("../concept_drift/tabela_final_2017_tres_anos_treinamento.csv")

numero_caracteristicas = enade_treino.shape[1] - 1
X_enade_treino = enade_treino.iloc[:, 0:numero_caracteristicas]
y_enade_treino = enade_treino.iloc[:, -1]

enade_teste = pd.read_csv("../concept_drift/tabela_final_2014_treinamento.csv")
#enade_teste = pd.read_csv("../concept_drift/tabela_final_2021_tres_anos_treinamento.csv")
#enade_teste = pd.read_csv("../pre-process_2021.csv")

In [34]:
train_dataset = Dataset(enade_treino, label='Nota_Conceito_Faixa', cat_features=[])
test_dataset = Dataset(enade_teste, label='Nota_Conceito_Faixa', cat_features=[])

In [35]:
check = MultivariateDrift(n_top_columns=10)
result = check.run(train_dataset=train_dataset, test_dataset=test_dataset, feature_importance_timeout=200)

In [36]:
result

In [10]:
dict_result = result.value

In [11]:
dict_result

{'domain_classifier_auc': 0.9707251520168685,
 'domain_classifier_drift_score': 0.941450304033737,
 'domain_classifier_feature_importance': {'nulos_Condicao_Salas': 0.6476831478125807,
  'nulos_Politica_de_Ingresso': 0.11778410561739584,
  'Todos': 0.11778410561739583,
  'nulos_Qtde_Livros': 0.11674864095262749,
  'Numero_Notas_Invalidas': 0.0,
  '40 horas ou mais': 0.0,
  'Critérios diferentes': 0.0,
  'nulos_Situacao_Trabalho': 0.0,
  'Não.1': 0.0,
  'Eventualmente': 0.0,
  'Até 20 horas': 0.0,
  '21 a 39 horas': 0.0,
  'nulos_Bolsa_de_Estudos': 0.0,
  'Bolsa de estudos': 0.0,
  'Sim': 0.0,
  'Não.2': 0.0,
  'Não se aplica.1': 0.0,
  'Nenhum': 0.0,
  '1 ou 2': 0.0,
  '3-5': 0.0,
  '6-8': 0.0,
  'Combinação de critérios': 0.0,
  'Critério de renda': 0.0,
  'nulos_Horas_de_Estudo': -0.0,
  'Critério étnico-racial': 0.0,
  'nulos_Moradia': 0.0,
  'Sozinho': 0.0,
  'Parentes': 0.0,
  'Cônjuge/Filhos': 0.0,
  'Outras pessoas': 0.0,
  'Universidade': 0.0,
  'Habitação individual/coletiva':

In [12]:
result_comparacao = pd.DataFrame()

result_comparacao["AUC score"] = [round(dict_result["domain_classifier_auc"], 3)]
result_comparacao["Drift score"] = [round(dict_result["domain_classifier_drift_score"], 3)]

In [13]:
result_comparacao

,AUC score,Drift score
0,0.971,0.941


In [9]:
result_comparacao.to_csv(f'./relatorio_drift/mlt_drift_17_21.csv',index=False)

In [35]:
dict_feature_importances = dict_result["domain_classifier_feature_importance"]

In [ ]:
dict_feature_importances

In [36]:
list_rank_feature_scores = sorted(dict_feature_importances.items(), key = lambda entry: entry[1])

labels = []
scores = []

for feature in list_rank_feature_scores[-10:]:
    labels.append(feature[0])
    scores.append(round(feature[1], 3))

print(labels)
print(scores)

['Concordo Total..38', 'Não se aplica.36', 'Não sei responder.37', 'Tem renda/Sustenta', 'EC', 'Prog CSF', 'Numero_Participantes', 'Disc. totalmente.38', 'Nenhum.2', 'Não sei responder.26']
[0.014, 0.017, 0.027, 0.035, 0.044, 0.062, 0.084, 0.109, 0.253, 0.356]


In [37]:
result_feature_importances = pd.DataFrame()

handle_latex_form = [
    feature_name.replace("_", "\_")
    for feature_name
    in labels
]

result_feature_importances["Valores"] = handle_latex_form
result_feature_importances["Importance score"] = scores
result_feature_importances

,Valores,Importance score
0,Concordo Total..38,0.014
1,Não se aplica.36,0.017
2,Não sei responder.37,0.027
3,Tem renda/Sustenta,0.035
4,EC,0.044
5,Prog CSF,0.062
6,Numero\_Participantes,0.084
7,Disc. totalmente.38,0.109
8,Nenhum.2,0.253
9,Não sei responder.26,0.356


In [272]:
result_feature_importances.to_csv(f'./relatorio_drift/mlt_drift_17_21_feat_imp.csv',index=False)